In [8]:
# this is week 4 test
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00


In [9]:
# Preamble
import sys
sys.path.append('..')

In [10]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

df_train = train_set.to_pandas()
df_val = validation_set.to_pandas()

print(len(df_train))
print(len(df_val))

df_train.head()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/116067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13325 [00:00<?, ? examples/s]

116067
13325


,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
2,వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,వేప,telugu,"{'answer_start': [12], 'answer_text': ['Azadir...","వేప (లాటిన్ Azadirachta indica, syn. Melia aza...",https://te.wikipedia.org/wiki/%E0%B0%B5%E0%B1%...
3,চেঙ্গিস খান কোন বংশের রাজা ছিলেন ?,চেঙ্গিজ খান,bengali,"{'answer_start': [414], 'answer_text': ['বোরজি...",চেঙ্গিজ খান (মঙ্গোলীয়: Чингис Хаан আ-ধ্ব-ব: ...,https://bn.wikipedia.org/wiki/%E0%A6%9A%E0%A7%...
4,రెయ్యలగడ్ద గ్రామ విస్తీర్ణత ఎంత?,రెయ్యలగడ్ద,telugu,"{'answer_start': [259], 'answer_text': ['27 హె...","రెయ్యలగడ్ద, విశాఖపట్నం జిల్లా, గంగరాజు మాడుగుల...",https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B1%...


In [11]:
# Get train and validation data for each language
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

df_val_bengali = df_val[df_val['language'] == 'bengali']
df_val_arabic = df_val[df_val['language'] == 'arabic']
df_val_indonesian = df_val[df_val['language'] == 'indonesian']


# For testing
df_train_english = df_train[df_train['language'] == 'english']
df_val_english = df_val[df_val['language'] == 'english']

In [12]:
def split_text_with_answer(row):
  start = row["annotations"]["answer_start"][0]

  if start == -1:
    return pd.Series([row["document_plaintext"], "", ""])
  end = row["annotations"]["answer_start"][0] + len(row["annotations"]["answer_text"][0])
  before = row["document_plaintext"][:start]
  after = row["document_plaintext"][end:]
  return pd.Series([before, row["annotations"]["answer_text"][0], after])

In [13]:
df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
df_train_english.head()

<ipython-input-13-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
<ipython-input-13-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
<ipython-input-13-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...


In [14]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00


In [15]:
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [16]:
def tokenize_before_after(row, tokenizer):
  before_tokens = tokenizer.tokenize(row["before"])
  answer_tokens = tokenizer.tokenize(row["answer"])
  after_tokens = tokenizer.tokenize(row["after"])
  return pd.Series([before_tokens, answer_tokens, after_tokens])

In [17]:
df_train_english[["before_tokens", "answer_tokens", "after_tokens"]] = df_train_english.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
df_train_english.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (985 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-17-7dda50543443>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before_tokens", "answer_tokens", "after_tokens"]] = df_train_english.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
<ipython-input-17-7dda50543443>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]"
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators..."


In [18]:
max_len = df_train_english.apply(lambda row: len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]), axis=1).max()
print(max_len)

4116


In [19]:
df_train_english["text_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len - (len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]))), axis=1)
df_train_english.head()

<ipython-input-19-087f3c0744ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["text_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len - (len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]))), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."


In [20]:
def create_BIO_tags_text(row):
  before = ["O"] * len(row["before_tokens"])
  if len(row["answer_tokens"]) == 0:
    answer = []
  else:
    answer = ["B"] + ["I"] * (len(row["answer_tokens"]) - 1)
  after = ["O"] * len(row["after_tokens"])
  pad = ["O"] * len(row["text_pad"])
  return before + answer + after + pad

In [21]:
df_train_english["BIO_tags_text"] = df_train_english.apply(create_BIO_tags_text, axis=1)
df_train_english.head()

<ipython-input-21-d523b1fb91be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["BIO_tags_text"] = df_train_english.apply(create_BIO_tags_text, axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ..."


In [22]:
df_lengts_not_eq = df_train_english[df_train_english.apply(lambda row: not (len(row["before_tokens"]) + len(row["answer_tokens"]) + len(row["after_tokens"]) + len(row["text_pad"]) == len(row["BIO_tags_text"])), axis=1)]
df_lengts_not_eq

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text


In [23]:
# tokenize question:
tokenizer = mbert_tokeniser
df_train_english["question_tokens"] = df_train_english.apply(lambda row: tokenizer.tokenize(row["question_text"]), axis=1)
df_train_english.head()

<ipython-input-23-f1d6a7642d2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["question_tokens"] = df_train_english.apply(lambda row: tokenizer.tokenize(row["question_text"]), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]"
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]"
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]"
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]"


In [24]:
max_len_questions = df_train_english.apply(lambda row: len(row["question_tokens"]), axis=1).max()
print(max_len_questions)

30


In [25]:
df_train_english["question_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len_questions - len(row["question_tokens"])), axis=1)
df_train_english.head()

<ipython-input-25-893fd88b38cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["question_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len_questions - len(row["question_tokens"])), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."


In [26]:
df_train_english["BIO_tags_question"] = df_train_english.apply(lambda row: ["O"] * max_len_questions, axis=1)
df_train_english.head()

<ipython-input-26-585ae4f82ce9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["BIO_tags_question"] = df_train_english.apply(lambda row: ["O"] * max_len_questions, axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [27]:
df_train_english["merged_BIO"] = df_train_english.apply(lambda row: row["BIO_tags_text"] + row["BIO_tags_question"], axis=1)
df_train_english.head()

<ipython-input-27-2b7725de0782>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["merged_BIO"] = df_train_english.apply(lambda row: row["BIO_tags_text"] + row["BIO_tags_question"], axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ..."


In [28]:
df_train_english["merged_tokens"] = df_train_english.apply(lambda row: row["before_tokens"] + row["answer_tokens"] + row["after_tokens"] + row["text_pad"] + row["question_tokens"] + row["question_pad"], axis=1)
df_train_english.head()

<ipython-input-28-dba44c17613e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["merged_tokens"] = df_train_english.apply(lambda row: row["before_tokens"] + row["answer_tokens"] + row["after_tokens"] + row["text_pad"] + row["question_tokens"] + row["question_pad"], axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[quantum, field, theory, naturally, began, wit..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[the, nobel, prize, in, literature, (, swedish..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[hangul, was, personally, created, and, promu,..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, 

In [29]:
df_lengts_not_eq = df_train_english[df_train_english.apply(lambda row: not (len(row["merged_BIO"])) == len(row["merged_tokens"]), axis=1)]
df_lengts_not_eq

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens


In [59]:
def BIO_tags_to_labels(row):
  # Create a dictionary to map the strings to numbers
  mapping = {"B": 2, "O": 0, "I": 1}

  # Use a list comprehension to convert the strings to numbers
  return [mapping[char] for char in row["merged_BIO"]]

In [60]:
df_train_english["BIO_labels"] = df_train_english.apply(BIO_tags_to_labels, axis=1)
df_train_english.head()

<ipython-input-60-c81798dbd965>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["BIO_labels"] = df_train_english.apply(BIO_tags_to_labels, axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens,BIO_labels
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[quantum, field, theory, naturally, began, wit...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[the, nobel, prize, in, literature, (, swedish...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[hangul, was, personally, created, and, promu,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect them

# Format of merged vector:
[before] + [answer] + [after] + [padding] + [question] + [padding]

In [36]:
!pip install bpemb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.7 MB/s eta 0:00:00


In [37]:
from bpemb import BPEmb

# Load English model with 25k word-pieces
bpemb_id = BPEmb(lang='eng', dim=100, vs=25000)

# Extract the embeddings
pretrained_embeddings = bpemb_id.emb.vectors

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs25000.model


100%|██████████| 661443/661443 [00:01<00:00, 554205.42B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs25000.d100.w2v.bin.tar.gz


100%|██████████| 9477142/9477142 [00:01<00:00, 4748175.91B/s]


# Following code is taken from lab 4

In [30]:
import io
from math import log
from numpy import array
from numpy import argmax
import torch
import random
from math import log
from numpy import array
from numpy import argmax
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR
from typing import List, Tuple, AnyStr
from tqdm.notebook import tqdm
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from copy import deepcopy
from datasets import load_dataset, load_metric
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import heapq

In [31]:
# Define the model
class BiLSTM(nn.Module):
    """
    Basic BiLSTM-CRF network
    """
    def __init__(
            self,
            pretrained_embeddings: torch.tensor,
            lstm_dim: int,
            dropout_prob: float = 0.1,
            n_classes: int = 2
    ):
        """
        Initializer for basic BiLSTM network
        :param pretrained_embeddings: A tensor containing the pretrained BPE embeddings
        :param lstm_dim: The dimensionality of the BiLSTM network
        :param dropout_prob: Dropout probability
        :param n_classes: The number of output classes
        """

        # First thing is to call the superclass initializer
        super(BiLSTM, self).__init__()

        # We'll define the network in a ModuleDict, which makes organizing the model a bit nicer
        # The components are an embedding layer, a 2 layer BiLSTM, and a feed-forward output layer
        self.model = nn.ModuleDict({
            'embeddings': nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1),
            'bilstm': nn.LSTM(
                pretrained_embeddings.shape[1],  # input size
                lstm_dim,  # hidden size
                2,  # number of layers
                batch_first=True,
                dropout=dropout_prob,
                bidirectional=True),
            'ff': nn.Linear(2*lstm_dim, n_classes),
        })
        self.n_classes = n_classes
        self.loss = nn.CrossEntropyLoss()
        # Initialize the weights of the model
        self._init_weights()

    def _init_weights(self):
        all_params = list(self.model['bilstm'].named_parameters()) + \
                     list(self.model['ff'].named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens, hidden_states = None, labels = None):
        """
        Defines how tensors flow through the model
        :param inputs: (b x sl) The IDs into the vocabulary of the input samples
        :param input_lens: (b) The length of each input sequence
        :param labels: (b) The label of each sample
        :return: (loss, logits) if `labels` is not None, otherwise just (logits,)
        """

        # Get embeddings (b x sl x edim)
        embeds = self.model['embeddings'](inputs)

        # Pack padded: This is necessary for padded batches input to an RNN - https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        # Pass the packed sequence through the BiLSTM
        if hidden_states:
            lstm_out, hidden = self.model['bilstm'](lstm_in, hidden_states)
        else:
            lstm_out, hidden = self.model['bilstm'](lstm_in)

        # Unpack the packed sequence --> (b x sl x 2*lstm_dim)
        lstm_out, lengths = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # Get logits (b x seq_len x n_classes)
        logits = self.model['ff'](lstm_out)
        outputs = (logits, lengths)
        if labels is not None:
            loss = self.loss(logits.reshape(-1, self.n_classes), labels.reshape(-1))
            outputs =  outputs + (loss,)

        return outputs

In [32]:
def train(
    model: nn.Module,
    train_dl: DataLoader,
    valid_dl: DataLoader,
    optimizer: torch.optim.Optimizer,
    n_epochs: int,
    device: torch.device,
    scheduler=None,
):
    """
    The main training loop which will optimize a given model on a given dataset
    :param model: The model being optimized
    :param train_dl: The training dataset
    :param valid_dl: A validation dataset
    :param optimizer: The optimizer used to update the model parameters
    :param n_epochs: Number of epochs to train for
    :param device: The device to train on
    :return: (model, losses) The best model and the losses per iteration
    """

  # Keep track of the loss and best accuracy
    losses = []
    learning_rates = []
    best_f1 = 0.0

    # Iterate through epochs
    for ep in range(n_epochs):

        loss_epoch = []

        #Iterate through each batch in the dataloader
        for batch in tqdm(train_dl):
            # VERY IMPORTANT: Make sure the model is in training mode, which turns on
            # things like dropout and layer normalization
            model.train()

            # VERY IMPORTANT: zero out all of the gradients on each iteration -- PyTorch
            # keeps track of these dynamically in its computation graph so you need to explicitly
            # zero them out
            optimizer.zero_grad()

            # Place each tensor on the GPU
            batch = tuple(t.to(device) for t in batch)
            input_ids = batch[0]
            seq_lens = batch[1]
            labels = batch[2]

            # Pass the inputs through the model, get the current loss and logits
            logits, lengths, loss = model(input_ids, seq_lens, labels=labels)
            losses.append(loss.item())
            loss_epoch.append(loss.item())

            # Calculate all of the gradients and weight updates for the model
            loss.backward()

            # Optional: clip gradients
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Finally, update the weights of the model
            optimizer.step()
            if scheduler != None:
                scheduler.step()
                learning_rates.append(scheduler.get_last_lr()[0])

        # Perform inline evaluation at the end of the epoch
        f1 = evaluate(model, valid_dl)
        print(f'Validation F1: {f1}, train loss: {sum(loss_epoch) / len(loss_epoch)}')

        # Keep track of the best model based on the accuracy
        if f1 > best_f1:
            torch.save(model.state_dict(), 'best_model')
            best_f1 = f1

    return losses, learning_rates

In [33]:


def evaluate(model: nn.Module, valid_dl: DataLoader):
    """
    Evaluates the model on the given dataset
    :param model: The model under evaluation
    :param valid_dl: A `DataLoader` reading validation data
    :return: The accuracy of the model on the dataset
    """
    # VERY IMPORTANT: Put your model in "eval" mode -- this disables things like
    # layer normalization and dropout
    model.eval()
    labels_all = []
    preds_all = []

    # ALSO IMPORTANT: Don't accumulate gradients during this process
    with torch.no_grad():
        for batch in tqdm(valid_dl, desc='Evaluation'):
            batch = tuple(t.to(device) for t in batch)
            input_ids = batch[0]
            seq_lens = batch[1]
            labels = batch[2]
            hidden_states = None

            logits, _, _ = model(input_ids, seq_lens, hidden_states=hidden_states, labels=labels)
            preds_all.extend(torch.argmax(logits, dim=-1).reshape(-1).detach().cpu().numpy())
            labels_all.extend(labels.reshape(-1).detach().cpu().numpy())

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds_all, average='macro')
    print(confusion_matrix(labels_all, preds_all))
    return F1



In [58]:


lstm_dim = 128
dropout_prob = 0.1
batch_size = 8
lr = 1e-2
n_epochs = 10
n_workers = 0  # set to a larger number if you run your code in colab

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

# Create the model
model = BiLSTM(
    pretrained_embeddings=torch.FloatTensor(pretrained_embeddings),
    lstm_dim=lstm_dim,
    dropout_prob=dropout_prob,
    n_classes=3
  ).to(device)



In [40]:
def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Combines multiple data samples into a single batch
    :param input_data: The combined input_ids, seq_lens, and labels for the batch
    :return: A tuple of tensors (input_ids, seq_lens, labels)
    """
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    # Pad all of the input samples to the max length (25000 is the ID of the [PAD] token)
    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    # Make sure each sample is max_length long
    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)

In [42]:
df_train_english.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[quantum, field, theory, naturally, began, wit..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[the, nobel, prize, in, literature, (, swedish..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[hangul, was, personally, created, and, promu,..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, 

In [43]:
df_train_english_merged = pd.DataFrame({
    'text':(df_train_english["merged_tokens"]),
    'answerable':(df_train_english["merged_BIO"])
    })

In [61]:
def pre_process_df(df):
  df[["before", "answer", "after"]] = df.apply(split_text_with_answer, axis=1)
  df[["before_tokens", "answer_tokens", "after_tokens"]] = df.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
  max_len = df.apply(lambda row: len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]), axis=1).max()
  df["text_pad"] = df.apply(lambda row: ["[PAD]"] * (max_len - (len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]))), axis=1)
  df["BIO_tags_text"] = df.apply(create_BIO_tags_text, axis=1)
  df["question_tokens"] = df.apply(lambda row: tokenizer.tokenize(row["question_text"]), axis=1)
  max_len_questions = df.apply(lambda row: len(row["question_tokens"]), axis=1).max()
  df["question_pad"] = df.apply(lambda row: ["[PAD]"] * (max_len_questions - len(row["question_tokens"])), axis=1)
  df["BIO_tags_question"] = df.apply(lambda row: ["O"] * max_len_questions, axis=1)
  df["merged_BIO"] = df.apply(lambda row: row["BIO_tags_text"] + row["BIO_tags_question"], axis=1)
  df["merged_tokens"] = df.apply(lambda row: row["before_tokens"] + row["answer_tokens"] + row["after_tokens"] + row["text_pad"] + row["question_tokens"] + row["question_pad"], axis=1)
  df["BIO_labels"] = df.apply(BIO_tags_to_labels, axis=1)
  res_df = pd.DataFrame({
    'text':(df["merged_tokens"]),
    'answerable':(df["BIO_labels"])
    })
  return df, res_df

In [62]:
english_train, english_train_merged = pre_process_df(df_train_english)

<ipython-input-61-1eb7cbc5bb49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["before", "answer", "after"]] = df.apply(split_text_with_answer, axis=1)
<ipython-input-61-1eb7cbc5bb49>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["before_tokens", "answer_tokens", "after_tokens"]] = df.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
<ipython-input-61-1eb7cbc5bb49>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [63]:
english_val, english_val_merged = pre_process_df(df_val_english)

<ipython-input-61-1eb7cbc5bb49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["before", "answer", "after"]] = df.apply(split_text_with_answer, axis=1)
<ipython-input-61-1eb7cbc5bb49>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["before_tokens", "answer_tokens", "after_tokens"]] = df.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
<ipython-input-61-1eb7cbc5bb49>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [64]:
train_dl = DataLoader(english_train_merged, batch_size=batch_size, shuffle=True, collate_fn=collate_batch_bilstm, num_workers=n_workers)
valid_dl = DataLoader(english_val_merged, batch_size=len(english_val_merged), collate_fn=collate_batch_bilstm, num_workers=n_workers)

# Create the optimizer
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CyclicLR(optimizer, base_lr=0., max_lr=lr, step_size_up=1, step_size_down=len(train_dl)*n_epochs, cycle_momentum=False)

# Train
losses, learning_rates = train(model, train_dl, valid_dl, optimizer, n_epochs, device, scheduler)
model.load_state_dict(torch.load('best_model'))

  0%|          | 0/924 [00:00<?, ?it/s]

KeyError: ignored